In [1]:
import pandas as pd
import numpy as np
import random
random.seed(42)
from datetime import datetime, timedelta
from sklearn import preprocessing
%load_ext autoreload
%autoreload 2
from GBRT_for_TSF.utils import evaluate_with_xgboost
from mpmf.utils import get_top_1_motif, get_top_1_motif_stumped, get_top_k_motifs, compute_point_after_average

In [2]:
def New_preprocessing(
    TimeSeries,
    num_periods_input,
    num_periods_output,
    include_itself,
    include_covariates,
    include_motif_information,
    k_motifs,
    no_points_after_motif,
):
    # print(len(TimeSeries))
    Data = []
    start_date = datetime(2012, 5, 1, 00, 00, 00)  # define start date
    for i in range(0, len(TimeSeries)):
        record = []
        record.append(TimeSeries[i])  # adding the pemds7 value
        record.append(start_date.month)
        record.append(start_date.day)
        record.append(start_date.hour)
        record.append(start_date.minute)
        record.append(start_date.weekday())
        record.append(start_date.timetuple().tm_yday)
        record.append(start_date.isocalendar()[1])
        start_date = start_date + timedelta(minutes=5)
        Data.append(record)
    ########## change list of lists to df ################
    headers = [
        "pems",
        "month",
        "day",
        "hour",
        "minute",
        "day_of_week",
        "day_of_year",
        "week_of_year",
    ]
    Data_df = pd.DataFrame(Data, columns=headers)
    sub = Data_df.iloc[:, 1:]
    # Normalize features to be from -0.5 to 0.5 as mentioned in the paper
    New_sub = preprocessing.minmax_scale(sub, feature_range=(-0.5, 0.5))
    Normalized_Data_df = pd.DataFrame(
        np.column_stack([Data_df.iloc[:, 0], New_sub]), columns=headers
    )
    if include_motif_information:
        if include_motif_information == 1:  # get_top_1_motif
            df_motif = get_top_1_motif_stumped(
                TimeSeries,
                num_periods_output,
                l=no_points_after_motif,
                include_itself=include_itself,
            )
            interested_features = [
                c for c in df_motif.columns if (("idx" not in c) and ("dist" not in c))
            ]
            df_motif = df_motif[interested_features]
        if include_motif_information == 2:  # get_top_k_motifs (Direct)
            df_motif = get_top_k_motifs(
                TimeSeries,
                num_periods_output,
                k=k_motifs,
                l=no_points_after_motif,
                include_itself=include_itself,
            )
            interested_features = [
                c for c in df_motif.columns if (("idx" not in c) and ("dist" not in c))
            ]
            df_motif = df_motif[interested_features]
        if include_motif_information == 3:  # get_top_k_motifs (Unweighted Average)
            df_motif = get_top_k_motifs(
                TimeSeries,
                num_periods_output,
                k=k_motifs,
                l=no_points_after_motif,
                include_itself=include_itself,
            )
            interested_features = [c for c in df_motif.columns if ("idx" not in c)]
            df_motif = df_motif[interested_features]
            df_motif = compute_point_after_average(df_motif)
        if include_motif_information == 4:  # get_top_k_motifs (Weighted Average)
            df_motif = get_top_k_motifs(
                TimeSeries,
                num_periods_output,
                k=k_motifs,
                l=no_points_after_motif,
                include_itself=include_itself,
            )
            interested_features = [c for c in df_motif.columns if ("idx" not in c)]
            df_motif = df_motif[interested_features]
            df_motif = compute_point_after_average(df_motif, method="weighted")
        # Normailize motif features to be from -0.5 to 0.5
        New_df_motif = preprocessing.minmax_scale(df_motif, feature_range=(-0.5, 0.5))
        # Convert the numpy array back to a DataFrame using the original columns and index
        New_df_motif = pd.DataFrame(
            New_df_motif, columns=df_motif.columns, index=df_motif.index
        )
        Normalized_Data_df = pd.concat([Normalized_Data_df, New_df_motif], axis=1)
    # print(Normalized_Data_df)
    #################################################################################################
    # cut training and testing training is 11232
    Train = Normalized_Data_df.iloc[0:11232, :]
    Train = Train.values
    Train = Train.astype("float32")
    # print('Traing length :',len(Train))
    Test = Normalized_Data_df.iloc[(11232 - num_periods_input) :, :]
    Test = Test.values
    Test = Test.astype("float32")
    # print('Traing length :',len(Test))
    # Number_Of_Features = 8
    Number_Of_Features = Normalized_Data_df.shape[1]
    ############################################ Windowing ##################################
    end = len(Train)
    start = 0
    next = 0
    x_batches = []
    y_batches = []
    while next + (num_periods_input + num_periods_output) < end:
        next = start + num_periods_input
        x_batches.append(Train[start:next, :])
        y_batches.append(Train[next : next + num_periods_output, 0])
        start = start + 1
    y_batches = np.asarray(y_batches)
    y_batches = y_batches.reshape(-1, num_periods_output, 1)
    x_batches = np.asarray(x_batches)
    x_batches = x_batches.reshape(-1, num_periods_input, Number_Of_Features)
    ############################################ Windowing ##################################
    end_test = len(Test)
    start_test = 0
    next_test = 0
    x_testbatches = []
    y_testbatches = []
    while next_test + (num_periods_input + num_periods_output) < end_test:
        next_test = start_test + num_periods_input
        x_testbatches.append(Test[start_test:next_test, :])
        y_testbatches.append(Test[next_test : next_test + num_periods_output, 0])
        start_test = start_test + 1
    y_testbatches = np.asarray(y_testbatches)
    y_testbatches = y_testbatches.reshape(-1, num_periods_output, 1)
    x_testbatches = np.asarray(x_testbatches)
    x_testbatches = x_testbatches.reshape(-1, num_periods_input, Number_Of_Features)

    if include_covariates and include_motif_information:
        pass
    elif include_covariates and (not include_motif_information):
        pass
    elif (not include_covariates) and include_motif_information:
        selected_cols = np.r_[
            0, len(headers) : len(headers) + len(df_motif.columns.tolist())
        ]
        x_batches = x_batches[:, :, selected_cols]
        x_testbatches = x_testbatches[:, :, selected_cols]
    else: # (not include_covariates) and (not include_motif_information)
        pass
    return x_batches, y_batches, x_testbatches, y_testbatches

In [3]:
file_name = "pems.npy"
data_path = r"../data/" + file_name
data = np.load(data_path)
data = pd.DataFrame(data)

num_periods_input = 9  # input
num_periods_output = 9  # to predict


# ================== Parameters that are going to be changed==================
include_covariates = False  # True/False : whether to include features or not
include_itself = True
include_motif_information = 1  # 0: no motif info; 1: Top-1 Motif; 2: Top-K Motifs (Direct); 3: Top-K Motifs (Average); 4: Top-K Motifs (Weighted Average)
k_motifs = 1  # 1, 3, 5
no_points_after_motif = 9  # number of points to consider: 1, ceil(m/2), m
# ================================================

xgboost_parameters = {
    "learning_rate": 0.045,
    "n_estimators": 150,
    "max_depth": 8,
    "min_child_weight": 1,
    "gamma": 0.0,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "scale_pos_weight": 1,
    "random_state": 42,
    "verbosity": 1,  # 0=Silent, 1=Warning, 2=Info, 3=Debug
}

# ALL_Test_Data = []
# ALL_Test_Prediction = []

In [4]:
# =================== Processing Time Series ===================
x_batches_Full = []
y_batches_Full = []
X_Test_Full = []
Y_Test_Full = []
for i in range(0, len(data)):
    print("Processing Time Series:", i)
    x_batches = []
    y_batches = []
    X_Test = []
    Y_Test = []
    TimeSeries = data.iloc[i, :]
    x_batches, y_batches, X_Test, Y_Test = New_preprocessing(
        TimeSeries,
        num_periods_input,
        num_periods_output,
        include_itself,
        include_covariates,
        include_motif_information,
        k_motifs,
        no_points_after_motif,
    )
    for element1 in x_batches:
        x_batches_Full.append(element1)

    for element2 in y_batches:
        y_batches_Full.append(element2)

    for element5 in X_Test:
        X_Test_Full.append(element5)

    for element6 in Y_Test:
        Y_Test_Full.append(element6)
## =================== End of Processing Time Series ===================
# Error 136: ValueError: could not broadcast input array from shape (8,) into shape (9,)

Processing Time Series: 0


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Processing Time Series: 1
Processing Time Series: 2
Processing Time Series: 3
Processing Time Series: 4
Processing Time Series: 5
Processing Time Series: 6
Processing Time Series: 7
Processing Time Series: 8
Processing Time Series: 9
Processing Time Series: 10
Processing Time Series: 11
Processing Time Series: 12
Processing Time Series: 13
Processing Time Series: 14
Processing Time Series: 15
Processing Time Series: 16
Processing Time Series: 17
Processing Time Series: 18
Processing Time Series: 19
Processing Time Series: 20
Processing Time Series: 21
Processing Time Series: 22
Processing Time Series: 23
Processing Time Series: 24
Processing Time Series: 25
Processing Time Series: 26
Processing Time Series: 27
Processing Time Series: 28
Processing Time Series: 29
Processing Time Series: 30
Processing Time Series: 31
Processing Time Series: 32
Processing Time Series: 33
Processing Time Series: 34
Processing Time Series: 35
Processing Time Series: 36
Processing Time Series: 37
Processing

/Users/cyuab/miniconda3/envs/mpmf/lib/python3.10/site-packages/stumpy/core.py:3467: UserWarning: A large number of values in `P` are smaller than 1e-06.
For a self-join, try setting `ignore_trivial=True`.
  warnings.warn(msg)


Processing Time Series: 142
Processing Time Series: 143
Processing Time Series: 144
Processing Time Series: 145
Processing Time Series: 146
Processing Time Series: 147
Processing Time Series: 148
Processing Time Series: 149
Processing Time Series: 150
Processing Time Series: 151
Processing Time Series: 152
Processing Time Series: 153
Processing Time Series: 154
Processing Time Series: 155
Processing Time Series: 156
Processing Time Series: 157
Processing Time Series: 158
Processing Time Series: 159
Processing Time Series: 160
Processing Time Series: 161
Processing Time Series: 162
Processing Time Series: 163
Processing Time Series: 164
Processing Time Series: 165
Processing Time Series: 166
Processing Time Series: 167
Processing Time Series: 168
Processing Time Series: 169
Processing Time Series: 170
Processing Time Series: 171
Processing Time Series: 172
Processing Time Series: 173
Processing Time Series: 174
Processing Time Series: 175
Processing Time Series: 176
Processing Time Seri

In [5]:
rmse, wape, mae, mape = evaluate_with_xgboost(
    num_periods_output,
    x_batches_Full,
    y_batches_Full,
    X_Test_Full,
    Y_Test_Full,
    xgboost_parameters,
    (include_covariates or (include_motif_information > 0)),
)
print("RMSE: ", rmse)
print("WAPE: ", wape)
print("MAE: ", mae)
print("MAPE: ", mape)

RMSE:  5.5815535
WAPE:  0.050957132
MAE:  2.9692013
MAPE:  0.07348905


In [6]:
import datetime

print(f"This notebook was last run end-to-end on: {datetime.datetime.now()}\n")

# motif = 9 points
# RMSE:  5.169188
# WAPE:  0.04775117
# MAE:  2.7823944
# MAPE:  0.06787598

This notebook was last run end-to-end on: 2026-01-10 16:19:52.626552

